In [1]:
import spacy
import string
import numpy as np
import itertools
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from alibi.explainers import AnchorText
from alibi.datasets import fetch_movie_sentiment
from alibi.utils.download import spacy_model
from alibi.utils.lang_model import DistilbertBaseUncased, BertBaseUncased, RobertaBase

%load_ext autoreload
%autoreload 2

/home/robert/anaconda3/envs/alibi/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


### Load movie review dataset

The `fetch_movie_sentiment` function returns a `Bunch` object containing the features, the targets and the target names for the dataset.

In [2]:
movies = fetch_movie_sentiment()
movies.keys()

dict_keys(['data', 'target', 'target_names'])

In [3]:
data = movies.data
labels = movies.target
target_names = movies.target_names

In [4]:
train, test, train_labels, test_labels = train_test_split(data, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

### Apply CountVectorizer to training set

In [5]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)

CountVectorizer()

### Fit model

In [6]:
np.random.seed(0)
clf = LogisticRegression(solver='liblinear')
clf.fit(vectorizer.transform(train), train_labels)

LogisticRegression(solver='liblinear')

### Define prediction function

In [7]:
predict_fn = lambda x: clf.predict(vectorizer.transform(x))

### Make predictions on train and test sets

In [8]:
preds_train = predict_fn(train)
preds_val = predict_fn(val)
preds_test = predict_fn(test)
print('Train accuracy', accuracy_score(train_labels, preds_train))
print('Validation accuracy', accuracy_score(val_labels, preds_val))
print('Test accuracy', accuracy_score(test_labels, preds_test))

Train accuracy 0.9801624284382905
Validation accuracy 0.7544910179640718
Test accuracy 0.7589841878294202


### Load spaCy model

English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.

In [9]:
model = 'en_core_web_md'
spacy_model(model=model)
nlp = spacy.load(model)

### Load transformers

In [10]:
lang_model = DistilbertBaseUncased()

### Random indices

In [11]:
indices = np.random.choice(len(test), size=10, replace=False)
print(indices)

[ 299  379  997 1601  264  229 1774 1968  458  252]


### Define Anchor

In [12]:
explainer = AnchorText(nlp=nlp, language_model=lang_model, predictor=predict_fn)

In [13]:
def build_explanation(explanation) -> str:
    s = ''
    s += 'Anchor: %s\n' % (' AND '.join(explanation.anchor))
    s += 'Precision: %.2f\n' % explanation.precision
    
    # print examples covered as True
    s += '\n\nExamples where anchor applies and model predicts %s:\n' % pred
    if len(explanation.raw['examples']):
        s += '\n'.join([x for x in explanation.raw['examples'][-1]['covered_true']])
    
    # print examples covered as False
    s += '\n\nExamples where anchor applies and model predicts %s:\n' % alternative
    if len(explanation.raw['examples']):
        s += '\n'.join([x for x in explanation.raw['examples'][-1]['covered_false']])
    
    s += '\n\n\n'
    return s

In [14]:
config = {
    "threshold": 0.95,
    "top_n": 50,
    "sample_proba": 0.5,
    "filling_method": 'parallel',
    "prec_mask_templates": 0.1,
    "punctuation": string.punctuation
}

In [15]:
class_names = movies.target_names

### Example 0

In [16]:
text = test[indices[0]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [17]:
print("Original text:", text)

Original text: a fantastically vital movie that manages to invest real humor , sensuality , and sympathy into a story about two adolescent boys .


In [18]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: manages AND real AND that
Precision: 0.97


Examples where anchor applies and model predicts positive:
each elegantly vital movie that manages to invest real anthology , sensuality , and compassion to this story well two adolescent juniors .
a fantastically necessitate filmmaker that manages to invest real anthology , sensuality , and outrage into a epilogue about two maternal teenagers .
a fantastically vital porno that manages to consider real humor , sensuality , and heartache into this backstory almost two adolescent daddies .
a fantastically satisfactory movie that manages to invest real impersonation , longing , and heartbreak into some heroine about two adolescent boys .
any fantastically vital movie that manages to realize real humor , sensuality , and defiance inside any story obviously two adoptive swimmers .
a surprisingly crucial foto that manages to privatize real humor , sensuality , and discontent like an protagonist only two adolescent boys .
a disturbingly effe

In [19]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: manages
Precision: 0.98


Examples where anchor applies and model predicts positive:
a fairy adult comic and manages to invest in humor, sadness, and sympathy in a story about two adolescent boys.
a novel novel strip who manages an invest in humor, wit, and sympathy in a story about two adolescent boys.
a teenage writer plot quartet manages to invest in humor, irony, and sympathy in a story about two adolescent boys.
a fictional and strip quartet manages to invest in humor, courage, and sympathy into a story about two adolescent boys.
a very drama fiction who manages to invest in humor, humor, and sympathy on a story about two adolescent boys.
a mysterious comic tale man manages to invest between humor, humor, and sympathy during a story about two adolescent boys.
a little suspense strip that manages to invest in humor, imagination, and sympathy in a story about two adolescent boys.
a teenage humor whose that manages to invest in humor, romance, and sympathy on a story about tw

### Example 1

In [20]:
text = test[indices[1]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

[W008] Evaluating Lexeme.similarity based on empty vectors.
[W008] Evaluating Lexeme.similarity based on empty vectors.


In [21]:
print("Original text:", text)

Original text: there is a refreshing absence of cynicism in stuart little 2--quite a rarity , even in the family film market . eventually , it wins you over .


In [22]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: refreshing AND family AND rarity AND film AND a AND of AND even
Precision: 0.96


Examples where anchor applies and model predicts positive:
there is another refreshing slowness of animus in swift difficult 2 - -quite a rarity , even through every family film trend . immediatly , it wins you over .
there is each refreshing certainty of immaturity in british sweet 2 - -quite a rarity , even in the family film consumer . again , it wins you over .
there is a refreshing unwillingness of cynicism like swift wonderful 2 - video a rarity , even towards those family film business . slowly , it wins you over .
there is a refreshing inconsistency of apathy in scottish little 2 - friend a rarity , even if the family film foundry . still , it sweeps you over .
there is a refreshing likelyhood of ignorance in stuart little 2 - -quite a rarity , even whether a family film equity . underway , it wins you over .
there is a refreshing absence of animus in stuart little 2 - -quite a rarity , ev

In [23]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: refreshing AND family AND you AND it AND film AND rarity
Precision: 0.97


Examples where anchor applies and model predicts positive:
there is a refreshing sense of cynicism in bbc babylon 2 - - quite a rarity, especially in the family film market. unfortunately, it gets you over.
there is a refreshing dose of cynicism in bad film 2 - - quite a rarity, especially in the family film market. however, it makes you over.
there is a refreshing sense of cynicism in paddington region 2 - - quite a rarity, especially in pakistani family film market. oh, it turns you over.
there is a refreshing bit of cynicism in happy movie 2 - - quite a rarity, particularly in telugu family film market. naturally, it gets you over.
there is a refreshing atmosphere of cynicism in bad picture 2 - - quite a rarity, especially in the family film market. hopefully, it wins you over.
there is a refreshing streak of cynicism in bbc playstation 2 - - quite a rarity, especially in russian family film market. s

### Example 2

In [24]:
text = test[indices[2]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

Could not find an result satisfying the 0.95 precision constraint. Now returning the best non-eligible result.


In [25]:
print("Original text:", text)

Original text: it doesn't offer audiences any way of gripping what its point is , or even its attitude toward its subject .


In [26]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: offer AND point AND audiences AND what AND toward AND its AND subject
Precision: 0.63


Examples where anchor applies and model predicts negative:
it does n't offer audiences those sense of gripping what its point is , or possibly its attitude toward its subject .
it does n't offer audiences some way under relaying what its point is , or possibly its mindset toward its subject .
it does n't offer audiences any way between gripping what its point is , or even its discipline toward its subject .
it does n't offer audiences some way for provoking what its point is , or possibly its zeal toward its subject .
it does n't offer audiences a part midst gripping what its point is , or already its piety toward its subject .
it does n't offer audiences these attention whether gripping what its point is , or sometimes its ethos toward its subject .
it does n't offer audiences any way as gripping what its point is , or maybe its belief toward its subject .
it does n't offer audiences any de

In [27]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: doesn AND point AND its AND offer AND audiences
Precision: 0.97


Examples where anchor applies and model predicts negative:
it doesn't offer audiences any way of gripping what any point represents, or controlling its contents about its subject.
it doesn't offer audiences any way of gripping what a point remains, or expanding its implications to its subject.
it doesn't offer audiences any way of gripping what is point takes, or changing its presentation about its subject.
it doesn't offer audiences any way of gripping what its point represents, or explaining its significance on its subject.
it doesn't offer audiences any way of gripping what its point lacks, or telling its appeal into its subject.
it doesn't offer audiences any way of gripping what viewers point involves, or turning its story around its subject.
it doesn't offer audiences any way of gripping what the point represents, or questioning its audience behind its subject.
it doesn't offer audiences any way of gripping

### Example 3

In [28]:
text = test[indices[3]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [29]:
print("Original text:", text)

Original text: rodriguez . . . was unable to reproduce the special spark between the characters that made the first film such a delight .


In [30]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: unable AND rodriguez AND characters
Precision: 0.95


Examples where anchor applies and model predicts negative:
rodriguez . . . was unable to transmit the perfect spark within the characters which made the exciting film such a bliss .
rodriguez . . . was unable to reproduce this special fire between the characters that made both important film such the delight .
rodriguez . . . was unable to exploit some available extinguisher beyond the characters that made the own footage such a delight .
rodriguez . . . was unable to alter another special spark with all characters that came the original poster such an relish .
rodriguez . . . was unable to reproduce the regular ignition between any characters whatever made these previous fable such a delight .
rodriguez . . . was unable to alter any special spark between some characters that made some other film such a heaven .
rodriguez . . . was unable to reproduce an special provocation between any characters whatever convinced the first

In [31]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: spark AND rodriguez AND unable AND made AND the
Precision: 0.97


Examples where anchor applies and model predicts negative:
rodriguez... definitely unable to contain the emotional spark through the characters that made the show become such as heroes.
rodriguez... deliberately unable to satisfy the magic spark from the characters that made the audience have such things memorable.
rodriguez... completely unable to remove the emotional spark from the characters that made the movie play such an events.
rodriguez... but unable to capture the drama spark beyond the characters that made the story make such many successes.
rodriguez... were unable to define the main spark or the characters that made the protagonist experience such frequent fail.
rodriguez... is unable to find the drama spark from the characters that made the comic develop such powerful entertainment.
rodriguez... purposely unable to produce the basic spark by the characters that made the villain play such passionate c

### Example 4

In [32]:
text = test[indices[4]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [33]:
print("Original text:", text)

Original text: while the story's undeniably hard to follow , iwai's gorgeous visuals seduce .


In [34]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: follow AND undeniably AND seduce
Precision: 0.96


Examples where anchor applies and model predicts positive:
while both story 's undeniably decent to follow , iwai 's gorgeous visuals seduce .
while the tragedy 's undeniably hard to follow , iwai 's gorgeous imaginations seduce .
while those epilogue 's undeniably ready to follow , iwai 's dashing graphics seduce .
while those memoir is undeniably hungry to follow , iwai 's gorgeous visuals seduce .
while these story loses undeniably hard to follow , iwai 's dashing textures seduce .
while the story believes undeniably hard to follow , iwai 's gorgeous visuals seduce .
while the fact understands undeniably strong to follow , iwai 's brilliant presentations seduce .
while this book sees undeniably big to follow , iwai 's sensational visuals seduce .
while the story 's undeniably black to follow , iwai 's magnificent visuals seduce .
while the cartoonist 's undeniably ready to follow , iwai 's dapper vocals seduce .

Examples wh

In [35]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: undeniably AND gorgeous
Precision: 0.99


Examples where anchor applies and model predicts positive:
while the story's undeniably hard to follow, zoe's gorgeous visuals seduce.
while the story's undeniably hard to follow, anastasia's gorgeous visuals seduce.
while each story's undeniably hard to follow, nichols's gorgeous visuals seduce.
while the story's undeniably hard to follow, james's gorgeous visuals seduce.
while another story's undeniably hard to follow, spears's gorgeous visuals seduce.
while the story's undeniably hard to follow, lewis's gorgeous visuals seduce.
while the story's undeniably hard to follow, roberts's gorgeous visuals seduce.
while each story's undeniably hard to follow, luke's gorgeous visuals seduce.
while the story's undeniably hard to follow, davies's gorgeous visuals seduce.
while its story's undeniably hard to follow, zoe's gorgeous visuals seduce.

Examples where anchor applies and model predicts negative:






### Example 5

In [36]:
text = test[indices[5]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [37]:
print("Original text:", text)

Original text: i have two words to say about reign of fire . great dragons !


In [38]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: dragons AND have AND fire
Precision: 0.95


Examples where anchor applies and model predicts negative:
i have two pronunciations to know about reign with fire . great dragons !
i have two words to understand that reign of fire . great dragons !
i have two things to bother around caliphate across fire . cool dragons !
i have two times to know about kingdom of fire . glad dragons !
i have two texts to suppose that figurehead of fire . great dragons !
i have two words to see about century of fire . great dragons !
i have two words to suppose about reign upon fire . extraordinary dragons !
i have two words to give past reign of fire . great dragons !
i have two meanings to disagree arround reign of fire . great dragons !
i have two words to say before vassal of fire . great dragons !

Examples where anchor applies and model predicts positive:
i have two words to realize about plutocracy of fire . great dragons !
i have two words to feel without caliphate of fire . happy dragons !
i

In [39]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: dragons AND say AND about
Precision: 0.97


Examples where anchor applies and model predicts negative:
i have two words to say about lots of dragons. great dragons!
we have two words to say about dragons of us. great dragons!
i have two words to say about gods of dragons. great dragons!
i have two words to say about lots of dragons. great dragons!
i have two words to say about plenty of dragons. great dragons!
dragons have two words to say about shades of dragons. great dragons!
we have two words to say about dragons of dragons. great dragons!
i have two words to say about monsters of dragons. great dragons!
we have two words to say about ser dragons castles. great dragons!
we have two words to say about dragons with dragons. great dragons!

Examples where anchor applies and model predicts positive:
must have two words to say about creatures of magic. great dragons!
i have two words to say about tales of elves. great dragons!
wizards have two words to say about our their dragon

### Example 6

In [40]:
text = test[indices[6]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [41]:
print("Original text:", text)

Original text: what's next : " my mother the car ? "


In [42]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: car AND next
Precision: 0.96


Examples where anchor applies and model predicts negative:
what 's next : " my mother this car ? "
what 's next : " my mother the car ? "
what 's next : " my mother the car ? "
what 's next : " my mother the car ? "
what 's next : " my mother those car ? "
what 's next : " my mother the car ? "
what 's next : " my grandmother every car ? "
what 's next : " my mother the car ? "
what 's next : " my mother the car ? "
what 's next : " my mother the car ? "

Examples where anchor applies and model predicts positive:
what 's next : " my baby the car ? "
what 's next : " my baby the car ? "





In [43]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: car
Precision: 0.99


Examples where anchor applies and model predicts negative:
o's next : " my mother the car? "
o's next : " my mother the car? "
jenkins's next : " my mother the car? "
i's next : " my mother the car? "
o's next : " my mother the car? "
o's next : " my mother the car? "
o's next : " my mother the car? "
o's next : " my mother the car? "
o's next : " my mother the car? "
p's next : " my mother the car? "

Examples where anchor applies and model predicts positive:
what's best : " you you my car? "





### Example 7

In [44]:
text = test[indices[7]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [45]:
print("Original text:", text)

Original text: supposedly based upon real , or at least soberly reported incidents , the film ends with a large human tragedy . alas , getting there is not even half the interest .


In [46]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: reported
Precision: 0.95


Examples where anchor applies and model predicts negative:
apparently calculated upon real , or at least soberly reported incidents , each anthology is with an sized human tragedy . alas , wanting there is not rather half some interest .
even associated upon real , or at least instinctively reported thefts , a supermodel ends between a tiny organism tragedy . alas , getting there is not even half the future .
supposedly based upon amazing , or behind fewest gleefully reported incidents , a repertoire edges under any large human catastrophe . alas , getting there is not possibly half these interest .
reportedly based upon same , or at least soberly reported bombings , the novella ends than a various human bystander . alas , getting there is not apparently half this interest .
litterally based upon real , or at fewest uncomfortably reported incidents , the film becomes from a large human uproar . alas , getting there is not even half each interest .
mis

In [47]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: getting AND large AND half AND the AND there
Precision: 0.98


Examples where anchor applies and model predicts negative:
supposedly based upon evidence, or by or or reported incidents, the episode ends with a large human tragedy. finally, getting there does hurts even half the participants.
supposedly based upon predictions, or after by often reported incidents, the game ends with a large human tragedy. unfortunately, getting there requires causes even half the tragedy.
supposedly based on reports, or other random or reported incidents, the festival ends with a large human tragedy. hence, getting there requires destroys even half the risk.
supposedly based on facts, or even and newly reported incidents, the story ends with another large human tragedy. however, getting there requires means even half the consequences.
supposedly based upon information, or rather circumstances many reported incidents, the book ends with a large human tragedy. fortunately, getting there is affects

### Example 8

In [48]:
text = test[indices[8]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [49]:
print("Original text:", text)

Original text: a well-crafted letdown .


In [50]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: crafted AND well AND a
Precision: 0.98


Examples where anchor applies and model predicts positive:
a well - crafted disdain .
a well - crafted incredulity .
a well - crafted concern .
a well - crafted angsty .
a well - crafted apathy .
a well - crafted letdown .
a well - crafted lament .
a well - crafted letdown .
a well - crafted letdown .
a well - crafted letdown .

Examples where anchor applies and model predicts negative:
a well - crafted embarrassment .
a well - crafted embarrassment .
a well - crafted embarrassment .
a well - crafted disappointment .





In [51]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: crafted AND well AND a AND letdown
Precision: 1.00


Examples where anchor applies and model predicts positive:
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.
a well - crafted letdown.

Examples where anchor applies and model predicts negative:






### Example 9

In [52]:
text = test[indices[9]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer.explain(
    text,
    sampling_method="similarity",
    **config
)

# language model explanation
lm_explanation = explainer.explain(
    text,
    sampling_method='language_model',
    **config
)

In [53]:
print("Original text:", text)

Original text: it's just hard to believe that a life like this can sound so dull .


In [54]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: dull
Precision: 1.00


Examples where anchor applies and model predicts negative:
it 's just hard to believe that a relationship without this can sound so dull .
it 's just hard to believe that a thing past this can sound not dull .
it 's just painful to beleive that a death around this can sing really dull .
it 's just hard to need that every life like some can sound so dull .
it 's just hard to believe that a family although an can sound only dull .
it 's just wrong to believe that a society although the can squeal well dull .
it 's somehow big to consider that the life like the can sound pretty dull .
it 's just nasty to believe that a world like this can sound so dull .
it 's so bad to doubt that a life like both can sound almost dull .
it 's just nasty to suppose that a life like this can hear so dull .

Examples where anchor applies and model predicts positive:






In [55]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: dull
Precision: 1.00


Examples where anchor applies and model predicts negative:
it's the you i believe that a life like ours can be so dull.
it's rare people you believe that a life like that can be so dull.
it's funny everyone anyone believe that a life like this can be so dull.
it's not to you believe that a life like this can look so dull.
it's the you everybody believe that a life like ours can be so dull.
it's funny time you believe that a life like marriage can be so dull.
it's not him you believe making a life like yours can be so dull.
it's odd the i believe that a life like ours can be so dull.
it's why make you believe that a life like yours can be so dull.
it's true him you believe that a life like this can get so dull.

Examples where anchor applies and model predicts positive:




